# **s15525_codes**

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder.appName("AirbnbPricePrediction").getOrCreate()

In [ ]:
# Load the Data into a Spark DataFrame
data = spark.read.csv("listings.csv", header=True, inferSchema=True)

In [ ]:
# Explore the Dataset
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- availability_365: integer (nullable = true)



In [ ]:
# print a portion of data
data.show(5)

+----+--------------------+-------+---------+-------------------+--------------+--------+----------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|host_name|neighbourhood_group| neighbourhood|latitude| longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+---------+-------------------+--------------+--------+----------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2318|Casa Madrona - Ur...|   2536|    Megan|       Central Area|       Madrona|47.61094|-122.29286|Entire home/apt|  475|            30|               32| 2020-02-01|             0.58|                           2.0|             238|
|6606|Fab, private seat...|  14942|    Joyce|Other neighborhoods

In [ ]:
data.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----------------+-------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------------------+------------------+
|summary|                  id|                name|             host_id|           host_name|neighbourhood_group|     neighbourhood|         latitude|          longitude|         room_type|             price|    minimum_nights|number_of_reviews|      last_review| reviews_per_month|calculated_host_listings_count|  availability_365|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----------------+-------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------------------+------------------+
|

In [ ]:
df = data.select("price", "number_of_reviews", "availability_365", "room_type")
df.show(5)

+-----+-----------------+----------------+---------------+
|price|number_of_reviews|availability_365|      room_type|
+-----+-----------------+----------------+---------------+
|  475|               32|             238|Entire home/apt|
|  102|              153|              87|Entire home/apt|
|   75|              149|             275|   Private room|
|  165|               45|             276|Entire home/apt|
|  125|               58|             311|Entire home/apt|
+-----+-----------------+----------------+---------------+
only showing top 5 rows



In [ ]:
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+
|summary|             price|number_of_reviews|  availability_365|         room_type|
+-------+------------------+-----------------+------------------+------------------+
|  count|              4096|             4096|              4093|              4096|
|   mean|179.24004884004884|62.47813339848522|180.32641094551673|       113.6756525|
| stddev|155.75809773574053|95.70068207997066| 115.6461826809035|175.64835677721658|
|    min|                 1|                0|                 0|        -122.29739|
|    max|   Entire home/apt|               99|               365|       Shared room|
+-------+------------------+-----------------+------------------+------------------+



In [ ]:
# drop rows with non numeric values in price variable
df = df.filter(col("price").rlike("^\d+(\.\d+)?$"))
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+
|summary|             price|number_of_reviews|  availability_365|         room_type|
+-------+------------------+-----------------+------------------+------------------+
|  count|              4095|             4095|              4092|              4095|
|   mean|179.24004884004884|62.48607038123167|180.36168132942328|192.33333333333334|
| stddev|155.75809773574053|95.71103055723518|115.63829715608804| 95.68873148565265|
|    min|                 1|                0|                 0|               117|
|    max|                99|               99|               365|       Shared room|
+-------+------------------+-----------------+------------------+------------------+



In [ ]:
# exact percentage of missing values in each column
from pyspark.sql.functions import col, count, when, isnan
total_count = df.count()
df.select([(count(when(isnan(c) | col(c).isNull(), c))/total_count).alias(c) for c in df.columns]).show()

+-----+-----------------+--------------------+---------+
|price|number_of_reviews|    availability_365|room_type|
+-----+-----------------+--------------------+---------+
|  0.0|              0.0|7.326007326007326E-4|      0.0|
+-----+-----------------+--------------------+---------+



In [ ]:
# drop missing values and make the datatypes correct
df = df.dropna()
df = df.withColumn("price", col("price").cast("int"))
df = df.withColumn("number_of_reviews", col("number_of_reviews").cast("int"))
df = df.withColumn("availability_365", col("availability_365").cast("int"))

In [ ]:
df.groupBy("room_type").count().show()

+---------------+-----+
|      room_type|count|
+---------------+-----+
|    Shared room|   41|
|Entire home/apt| 3382|
|   Private room|  669|
+---------------+-----+



In [ ]:
# encode room_type using one hot encoding and drop the original
indexer = StringIndexer(inputCol="room_type", outputCol="room_type_index")
indexed_df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCols=["room_type_index"], outputCols=["room_type_vec"], dropLast=True)
encoded_df = encoder.fit(indexed_df).transform(indexed_df)
encoded_df.show()

+-----+-----------------+----------------+---------------+---------------+-------------+
|price|number_of_reviews|availability_365|      room_type|room_type_index|room_type_vec|
+-----+-----------------+----------------+---------------+---------------+-------------+
|  475|               32|             238|Entire home/apt|            0.0|(2,[0],[1.0])|
|  102|              153|              87|Entire home/apt|            0.0|(2,[0],[1.0])|
|   75|              149|             275|   Private room|            1.0|(2,[1],[1.0])|
|  165|               45|             276|Entire home/apt|            0.0|(2,[0],[1.0])|
|  125|               58|             311|Entire home/apt|            0.0|(2,[0],[1.0])|
|   88|               76|             311|Entire home/apt|            0.0|(2,[0],[1.0])|
|   75|               43|             242|   Private room|            1.0|(2,[1],[1.0])|
|   82|              857|              99|Entire home/apt|            0.0|(2,[0],[1.0])|
|   67|              

In [ ]:
# drop room_type and room_type_index columns
encoded_df = encoded_df.drop("room_type")
encoded_df = encoded_df.drop("room_type_index")
encoded_df.show()

+-----+-----------------+----------------+-------------+
|price|number_of_reviews|availability_365|room_type_vec|
+-----+-----------------+----------------+-------------+
|  475|               32|             238|(2,[0],[1.0])|
|  102|              153|              87|(2,[0],[1.0])|
|   75|              149|             275|(2,[1],[1.0])|
|  165|               45|             276|(2,[0],[1.0])|
|  125|               58|             311|(2,[0],[1.0])|
|   88|               76|             311|(2,[0],[1.0])|
|   75|               43|             242|(2,[1],[1.0])|
|   82|              857|              99|(2,[0],[1.0])|
|   67|               97|             286|(2,[0],[1.0])|
|   95|               28|             194|(2,[0],[1.0])|
|  149|                0|             123|(2,[0],[1.0])|
|  112|              181|             139|(2,[0],[1.0])|
|   75|               45|             285|(2,[0],[1.0])|
|  100|              143|             294|(2,[1],[1.0])|
|   89|              426|      

In [ ]:
# remove duplicate observations
encoded_df = encoded_df.dropDuplicates()

In [ ]:
encoded_df.show()

+-----+-----------------+----------------+-------------+
|price|number_of_reviews|availability_365|room_type_vec|
+-----+-----------------+----------------+-------------+
|  321|               83|              37|(2,[0],[1.0])|
|  224|              118|             219|(2,[0],[1.0])|
|  138|               93|             122|(2,[0],[1.0])|
|   79|              117|             257|(2,[0],[1.0])|
|  110|              227|               4|(2,[0],[1.0])|
|  204|              266|             141|(2,[0],[1.0])|
|  229|              230|             251|(2,[0],[1.0])|
|   85|                4|             365|(2,[1],[1.0])|
|   70|                3|             263|(2,[0],[1.0])|
|  199|               53|             180|(2,[0],[1.0])|
|  147|               13|             315|(2,[0],[1.0])|
|  130|               57|             262|(2,[0],[1.0])|
|  135|              134|             265|(2,[0],[1.0])|
|  140|               62|              26|(2,[0],[1.0])|
|  112|                0|      

In [ ]:
# Assemble features into a single vector
assembler = VectorAssembler(inputCols=["number_of_reviews", "availability_365", "room_type_vec"], outputCol="features")
df_final = assembler.transform(encoded_df).select("features", "price")

In [ ]:
# split the df_new into training and testing to predict the price
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# **Fit several models, evaluate them and find the best one**

In [ ]:
from pyspark.ml.regression import (LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor, GeneralizedLinearRegression)
from pyspark.ml.evaluation import RegressionEvaluator
from xgboost.spark import SparkXGBRegressor

In [ ]:
# Initialize models
models = [
    ("Linear Regression", LinearRegression(featuresCol='features', labelCol='price')),
    ("Decision Tree", DecisionTreeRegressor(featuresCol='features', labelCol='price')),
    ("Random Forest", RandomForestRegressor(featuresCol='features', labelCol='price')),
    ("Gradient-Boosted Tree", GBTRegressor(featuresCol='features', labelCol='price')),
    ("Generalized Linear Regression", GeneralizedLinearRegression(featuresCol='features', labelCol='price')),
    ("XGBoost Regressor", SparkXGBRegressor(features_col='features', label_col='price'))
    ]

In [ ]:
# Fit models and make predictions
results = []
for name, model in models:
    fitted_model = model.fit(train_data)
    predictions = fitted_model.transform(test_data)
    results.append((name, predictions))

INFO:XGBoost-PySpark:Running xgboost-2.1.1 on 1 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!


In [ ]:
# Initialize evaluator
evaluator = RegressionEvaluator(labelCol='price', predictionCol='prediction')

# Evaluate models
metrics = []
for name, predictions in results:
    rmse = evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'})
    r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
    metrics.append((name, rmse, r2))

In [ ]:
# print results
import pandas as pd
metrics_df = pd.DataFrame(metrics, columns=['Model', 'RMSE', 'R2'])
metrics_df

,Model,RMSE,R2
0,Linear Regression,139.10,0.12
1,Decision Tree,138.33,0.13
2,Random Forest,137.57,0.14
3,Gradient-Boosted Tree,139.26,0.12
4,Generalized Linear Regression,139.10,0.12
5,XGBoost Regressor,179.18,-0.46


In [ ]:
# the best model
best_model = min(metrics, key=lambda x: x[1])
print(f"The best model is {best_model[0]} with RMSE = {best_model[1]} and R^2 = {best_model[2]}")

The best model is Random Forest with RMSE = 137.56727537208025 and R^2 = 0.1411650930272612
